# Exploration de données dans un notebook

In [ ]:
# installation d'un nouveau package
! pip install azure-storage-blob

In [ ]:
# import des packages nécessaires pour le traitement
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from azure.storage.blob import BlockBlobService

In [ ]:
#Get Path for the current folder position and add the following path in parameter.
current_path = 'C:/Users/PaulPeton/OneDrive - AZEO/Meetups/Citibike/dataset/JC'
globalpath = os.getcwd()+current_path
extension = 'csv'

#List all files in 'globalpath' and for each files ending with 'extension',
# parameter and store the value in 'listfiles'.
list_files = [f for f in os.listdir(current_path) if f.endswith(f'.{extension}')]
print(list_files)

In [ ]:
# lecture des fichiers d'un compte de stockage Azure
block_blob_service = BlockBlobService(
    account_name='nycitibikepubstr', account_key='PaP0sDTVDgYpqorHlODYtGhJgQfrZAOR+ZVTOAZdk7EODMUETeeep6oCkO6L0qshFoibB4kbhKbtY4SPVDSA6A==')

container_URL = 'https://nycitibikepubstr.blob.core.windows.net/csv'
container_name = 'jccsv'

In [ ]:
# Liste des éléments présents dans le container
print("\nList blobs in the container")
generator = block_blob_service.list_blobs(container_name)
list_files = []

for blob in generator:
    print("\t Blob name: " + blob.name)
    print("\t File URL: https://nycitibikepubstr.blob.core.windows.net/jccsv/" + blob.name )
    list_files.append("https://nycitibikepubstr.blob.core.windows.net/jccsv/" + blob.name)

In [ ]:
list_files

In [ ]:
# chargement des données externes dans des objets de type pandas DataFrame
df = pd.read_csv('https://nycitibikepubstr.blob.core.windows.net/jccsv/JC-201901-citibike-tripdata.csv')

In [ ]:
df.shape

In [ ]:
col_names = ['tripduration','starttime','stoptime','start_station_id','start_station_name','start_station_latitude','start_station_longitude','end_station_id','end_station_name','end_station_latitude','end_station_longitude','bikeid','usertype','year_of_birth','gender']

In [ ]:
# ne marche pas depuis une URL Azure
import glob

filenames = glob.glob('https://nycitibikepubstr.blob.core.windows.net/jccsv/JC_*.csv')
print(filenames)

In [ ]:
df = pd.DataFrame() #columns = col_names
total_length = 0

for file in list_files:
    single_df = pd.read_csv(file) #skiprows=[1], header=None
    print(single_df.columns)
    print(single_df.shape[0])
    total_length = total_length + single_df.shape[0]
    df = df.append(single_df)

print(total_length)

#details = "Nombre de lignes des fichiers : {0} + {1} + {2}".format(data_m1.shape[0], data_m2.shape[0], data_m3.shape[0])
total = "\tVérification du total : {}".format(df.shape[0])
print(total)

In [ ]:
# renommage des colonnes du dataframe
df.columns = col_names

In [ ]:
# affichage des premières lignes d'un DataFrame
df.head(10)

In [ ]:
# affichage du nombre de valeurs et des types de colonnes au sein de l'objet DataFrame
df.info()

In [ ]:
# convertion en dates et vérification du nouveau types des colonnes 
df['starttime'] = pd.to_datetime(df['starttime'], format='%Y-%m-%d %H:%M:%S')
df['stoptime'] = pd.to_datetime(df['stoptime'], format='%Y-%m-%d %H:%M:%S')
df.dtypes

In [ ]:
# nouveau contrôle du nombre de valeurs manquantes
df.isnull().sum()

In [ ]:
# description statistique des variables (seulement numériques)
df[['tripduration','year_of_birth']].describe()

In [ ]:
# durées aberrantes ?
df[(df['tripduration']>59) & (df['tripduration']<50*60)].head()

In [ ]:
# histogramme des durées de trajet
%matplotlib inline
plt.hist('tripduration', bins = 100,  data = df[(df['tripduration']>59) & (df['tripduration']<50*60)])
plt.show()

In [ ]:
# On vérifie si la station de départ est différente de la station d'arrivée
df[df['start_station_id'] == df['end_station_id']].head()

In [ ]:
# On créer un DataFrame nettoyé
df_clean = df[(df['tripduration']>59) & (df['tripduration']<50*60)]
print(df_clean.shape)

df_clean = df_clean[df_clean['start_station_id'] != df_clean['end_station_id']]
print(df_clean.shape)

df_clean = df_clean[df_clean['year_of_birth'] != 1888]
print(df_clean.shape)

In [ ]:
# comptage des valeurs pour une variable catégorielle
df['gender'].value_counts()

In [ ]:
# fréquences relatives pour une variable catégorielle
df['gender'].value_counts(normalize=True) #.mul(100).round(1).astype(str) + '%'

In [ ]:
df = df.drop('genderc', axis=1)

In [ ]:
# Renommer les libellés : Zero=unknown; 1=male; 2=female
#df['genderc'] = df['gender'].astype("category", ordered = False, labels = ['Unknown','Male','Female'])

gender_dict = {'gender': {0: 'unknown', 1: 'male', 2: 'female'}}
df['genderc'] = df['gender'].map(gender_dict['gender'])
df[['gender','genderc']].head()

In [ ]:
# convertir les catégories en nombres
# http://benalexkeen.com/mapping-categorical-data-in-pandas/
df['gendern'] = df['genderc'].astype("category", ordered = False, labels = ['Unknown','Male','Female'])
df['gendern'].head().cat.codes

In [ ]:
# Opérations inutiles, on supprime les deux colonnes
df.drop(['genderc','gendern'], axis=1, inplace=True)
df.columns

In [ ]:
# transformer les données 'birth year' et 'gender'
df['age'] = 2019 - df['year_of_birth']
df[['year_of_birth','age']].head(10)

In [ ]:
df['age'].describe()

In [ ]:
df[df['age']>99]

In [ ]:
# calcul de la distance à vol d'oiseau
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

def distance(row):
    lat1 = radians(row['start station latitude'])
    lon1 = radians(row['start station longitude'])
    lat2 = radians(row['end station latitude'])
    lon2 = radians(row['end station longitude'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [ ]:
# ajout de la colonne distance
df['distance'] = df[['start station latitude','start station longitude','end station latitude','end station longitude']].apply(distance, axis = 1)

In [ ]:
# temps moyen de trajet par genre
df[['tripduration','gender']].groupby(['gender']).agg(['mean', 'count', 'min', 'max'])

In [ ]:
start_stations = df['start_station_id'].unique()
print(sorted(start_stations))

In [ ]:
end_stations = df['end_station_id'].unique()
print(sorted(end_stations))

In [ ]:
start_stations.tolist().symmetric_difference(end_stations.tolist())

In [ ]:
np.setdiff1d(start_stations, end_stations)

In [ ]:
np.setdiff1d(end_stations, start_stations)

In [ ]:
start_end_combinaisons = df.groupby(['start_station_id','end_station_id'], as_index=False)['bikeid'].count()
# renommer la colonne bikeid
start_end_combinaisons.columns = ['start','end','count']


In [ ]:
start_end_combinaisons.sort_values(by=['count'], ascending=False)

In [ ]:
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
start_agg = df.groupby(
   ['start_station_id']
).agg(
    {
        'start_station_id': 'count', # get the count of rows 
        'bikeid' : 'nunique', # get the count of unique values
        'tripduration': lambda x : (x - 60).mean(), # mean duration per group
        'age': ['mean', 'median', min, max]  # get the mean of age
    }
)

In [ ]:
start_agg = df.groupby(
   ['start_station_id'] # ajouter lat / long end station
).agg(
        {
        'start_station_id' : [('row_count', 'count')],  # get the count of rows 
        'bikeid' : [('distinct_bike','nunique')], # get the count of unique values
        'tripduration' : [('tripinminuts',lambda x : (x / 60).mean())], # mean duration per group
        'age': [('age_median','median'), ('age_mean','mean'), ('age_std','std'), ('age_min','min'), ('age_max','max')]
        }
)

In [ ]:
start_agg.head(10)

In [ ]:
start_agg.columns.values.tolist()

In [ ]:
# le résultat est un dataframe
start_agg['tripduration'] = start_agg['tripduration'].round(0)
start_agg[('age','age_mean')] = start_agg[('age','age_mean')].round(0)
start_agg[('age','age_std')] = start_agg[('age','age_std')].round(1)
start_agg.head()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=0).fit(start_agg)
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.predict([[9999,600,40]])

In [ ]:
start_agg['cluster'] = pd.Series(kmeans.labels_, index=start_agg.index)
# convertir en catégorie
start_agg.sort_values(['cluster']).head()

In [ ]:
start_agg['cluster'].value_counts().sort_index()

In [ ]:
start_agg[start_agg['cluster']==0].describe()
# boucler sur chaque cluster

In [ ]:
classes_km = kmeans.labels_
plt.figure(figsize=(12,7))

markers=["+","s","^","v","x"]
for val, mark in zip(np.unique(classes_km), markers):
    plt.scatter(start_agg['end station latitude'][classes_km==val], start_agg['end station longitude'][classes_km==val], \
    marker = mark, label = "classe % i"%(val))
plt.title("Clusters on end stations")
plt.legend()

In [ ]:
df['startdate'] = df['starttime'].astype('M8[D]')
df['stopdate'] = df['stoptime'].astype('M8[D]')
df['startmonth'] = df['starttime'].apply(lambda x : x.month)

In [ ]:
df['startmonth'].value_counts()

In [ ]:
## SERIE TEMPORELLE
ts = df.groupby(
   ['startmonth','startdate']
).agg(
    {
        'bikeid': 'count', # get the count of rows 
    })

In [ ]:
# renommer la colonne

ts.head()

In [ ]:
ts.plot()

In [ ]:
df_ts = df.groupby(['startmonth','startdate']).size()
df_ts = df_ts.reset_index()
# renommer la dernière colonne
df_ts.rename(columns={ df_ts.columns[2]: "sharing" }, inplace = True)
df_ts.head()

In [ ]:
# https://towardsdatascience.com/subplots-in-matplotlib-a-guide-and-tool-for-planning-your-plots-7d63fa632857
fig, axs = plt.subplots(3, 1, figsize=(15, 9), sharey=True)

axs[0].plot(df_ts[df_ts['startmonth']==1]["startdate"], df_ts[df_ts['startmonth']==1]["sharing"])
axs[1].plot(df_ts[df_ts['startmonth']==2]["startdate"], df_ts[df_ts['startmonth']==2]["sharing"])
axs[2].plot(df_ts[df_ts['startmonth']==3]["startdate"], df_ts[df_ts['startmonth']==3]["sharing"])

fig.suptitle('Monthly subplot')

In [ ]:
### PREVISION TEMPORELLE (autoML)

In [ ]:
### REGRESSION DE LA DUREE DU TRAJET

In [ ]:
## régression linéaire

In [ ]:
# scatter plot distance x duration
df_clean.plot(kind="scatter", x="distance", y="tripduration", alpha=0.4)
plt.show()

In [ ]:
x = df_clean[['distance','age']]
y = df_clean['tripduration']
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# fit (train)
from sklearn import linear_model
linmodel = linear_model.LinearRegression(fit_intercept=True, normalize=False)
linmodel.fit(x_train, y_train)

In [ ]:
# predict
y_predict = linmodel.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#from sklearn.metrics import explained_variance_score
from sklearn.metrics import median_absolute_error

print("MCE : ", mean_squared_error(y_test,y_predict))
print("R2 : ", r2_score(y_test,y_predict))
print("RMCE : ", np.sqrt(mean_squared_error(y_test,y_predict)))
print("MAE : ", median_absolute_error(y_test,y_predict))

In [ ]:
# scatter plot des valeurs réelles vs prédites

plt.scatter(x_test['distance'], y_test,  color='black')
plt.plot(x_test['distance'], y_predict, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
## random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=100)
rfmodel.fit(x_train, y_train)
y_predict = rfmodel.predict(x_test)

print("MCE : ", mean_squared_error(y_test,y_predict))
print("R2 : ", r2_score(y_test,y_predict))
print("RMCE : ", np.sqrt(mean_squared_error(y_test,y_predict)))
print("MAE : ", median_absolute_error(y_test,y_predict))

In [ ]:
# validation croisée
scores = cross_validate(linmodel, x_train, y_train, cv=3, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
print(scores['test_neg_mean_squared_error']) 

In [ ]:
# recherche des meilleurs hyperparamètres
from sklearn.model_selection import GridSearchCV
dico_param = {'alpha': [1e-3, 1e-2, 1e-1, 1]}
search_hyperp_ridge = GridSearchCV(Ridge(), dico_param, scoring='neg_mean_squared_error', cv = 5)
search_hyperp_ridge.fit(x_train, x_train)
search_hyperp_ridge.predict(x_test)

print(search_hyperp_ridge.best_params_)
print(search_hyperp_ridge.best_score_)

In [ ]:
## exporter le meilleur modèle au format pickle

In [ ]:
# enregistrer le modèle au format pickle

from sklearn.externals import joblib
joblib.dump(linmodel, 'lin_model.pkl')
joblib.dump(rfmodel, 'rf_model.pkl')

In [ ]:
# enregistrer le modèle dans Azure Blob Storage

block_blob_service = BlockBlobService(
    account_name='nycitibikepubstr', account_key='XXX')

container_URL = 'https://nycitibikepubstr.blob.core.windows.net/csv'
container_name = 'pkl'

local_path = os.path.expanduser("~\Documents")
local_file_name = "lin_model.pkl"
full_path_to_file = os.path.join(local_path, local_file_name)

block_blob_service.create_blob_from_path(
    container_name, local_file_name, full_path_to_file)
